# Synopsis-to-Vector

In [1]:
import numpy as np
import pandas as pd
import pickle
import math

## Load Pre-trained CBOW Parameters

In [2]:
pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

## Get TF-IDF from synopsis.csv

### Load Cleaned Synopsis Data

In [3]:
synopsis = pd.read_csv('../clean_data/synopsis.csv')
synopsis.head()

,MAL_ID,Synopsis
0,1,solar leaving surface planet earth solar polic...
1,5,day life crew routine interrupted chasing targ...
2,6,head reason waste oppose entire cities fun tit...
3,7,individuals powers mind control robin craft us...
4,8,dark century people suffering rule manipulate ...


### Get Word Set from Synopsis Data

In [4]:
syns = synopsis.Synopsis.str.findall('\w+')

vocab = set()
for syn in syns:
    vocab.update(syn)

vocab = sorted(list(vocab))
vocab_size = len(vocab)

### Get TF(Term Frequency)

In [5]:
def get_tf(word, syn):
    # Term Frequency
    return syn.count(word)

    
tf = []
for syn in syns:
    tf_syn = []
    for word in vocab:
        tf_syn.append(get_tf(word, syn))
    tf.append(tf_syn)

tf = np.asarray(tf)
tf.shape

(6000, 5921)

### Get IDF(Inverse Document Frequency)

In [6]:
def get_idf(term):
    # Inverse Document Frequency
    _df = 0
    for syn in syns:
        _df += int(word in syn)
    return math.log(vocab_size/(_df))


idf = []
for word in vocab:
    idf.append(get_idf(word))

idf = np.asarray(idf)
idf.shape

(5921,)

### Get TF-IDF Table

In [7]:
tf_idf = pd.DataFrame(tf * idf, columns=vocab)

## Generate Sentence Vector of Synopsis

In [8]:
def get_word_vec(word):
    return word_vecs[word_to_id[word]]

def get_tf_idf(word, i):
    return tf_idf.loc[i, word]

def get_sent_vec(i):
    sent_vec = np.zeros(100, dtype=np.float16)

    for word in set(syns[i]):
        sent_vec += get_word_vec(word) * get_tf_idf(word, i)
    
    return sent_vec / len(set(syns[i]))

In [9]:
full_sent_vec = []
for i in range(len(syns)):
    full_sent_vec.append(get_sent_vec(i))
    
sent_vec_df = pd.DataFrame(full_sent_vec)
sent_vec_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.115295,-3.925781,0.509277,1.326172,2.685547,1.787109,-1.766602,-1.253906,-1.898438,1.073242,...,0.862305,-0.738770,-1.120117,-0.728516,1.066406,1.273438,-0.247070,1.378906,1.921875,2.218750
1,2.314453,-2.363281,-0.013374,2.705078,1.199219,3.171875,-1.359375,-2.605469,-1.960938,-0.565430,...,0.495850,-0.945801,-0.938965,-0.259766,2.087891,1.500000,-0.221924,0.645508,2.220703,0.959473
2,-0.225220,-2.599609,-0.183350,2.042969,1.671875,1.951172,-1.367188,-2.916016,-0.161621,0.050690,...,-1.436523,-1.022461,-2.535156,-2.455078,2.013672,1.943359,0.623047,1.701172,2.359375,1.232422
3,0.844727,-0.527344,0.271484,0.753418,1.576172,2.492188,-1.652344,-2.705078,-2.271484,1.469727,...,-0.366943,-1.791016,-1.804688,-2.474609,2.880859,0.426514,-0.582520,3.105469,0.839355,1.042969
4,1.522461,-2.654297,0.423584,1.816406,2.029297,1.697266,0.608887,-1.333008,-1.022461,0.936035,...,1.652344,-1.703125,-2.476562,-4.488281,1.402344,0.639160,0.059204,1.250000,2.458984,2.517578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.907227,-1.423828,1.901367,2.382812,1.664062,0.900879,-1.924805,-0.843262,-1.282227,1.338867,...,1.071289,-0.565918,-1.356445,-2.285156,0.418701,1.647461,0.828125,1.503906,2.166016,2.939453
5996,2.138672,-1.450195,0.352051,2.126953,0.429688,-0.171875,-0.984863,-2.195312,-2.027344,1.022461,...,-0.836914,-1.895508,-2.447266,-2.021484,1.229492,0.562500,-1.253906,2.990234,2.492188,1.022461
5997,1.147461,-2.382812,0.937988,0.431396,1.119141,2.527344,-1.699219,-1.219727,-1.711914,1.555664,...,0.713379,-0.082031,-1.974609,-0.666992,2.468750,0.973633,1.822266,-0.182739,2.941406,0.713867
5998,0.945801,-1.449219,-0.097046,0.824707,1.160156,1.100586,-0.375732,-2.132812,-0.812012,2.246094,...,1.913086,0.848145,-1.116211,-1.067383,1.034180,1.029297,-0.780762,0.202637,2.699219,1.937500


In [10]:
sent_vec_df.insert(0, 'MAL_ID', synopsis.MAL_ID)
sent_vec_df

,MAL_ID,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,1,-0.115295,-3.925781,0.509277,1.326172,2.685547,1.787109,-1.766602,-1.253906,-1.898438,...,0.862305,-0.738770,-1.120117,-0.728516,1.066406,1.273438,-0.247070,1.378906,1.921875,2.218750
1,5,2.314453,-2.363281,-0.013374,2.705078,1.199219,3.171875,-1.359375,-2.605469,-1.960938,...,0.495850,-0.945801,-0.938965,-0.259766,2.087891,1.500000,-0.221924,0.645508,2.220703,0.959473
2,6,-0.225220,-2.599609,-0.183350,2.042969,1.671875,1.951172,-1.367188,-2.916016,-0.161621,...,-1.436523,-1.022461,-2.535156,-2.455078,2.013672,1.943359,0.623047,1.701172,2.359375,1.232422
3,7,0.844727,-0.527344,0.271484,0.753418,1.576172,2.492188,-1.652344,-2.705078,-2.271484,...,-0.366943,-1.791016,-1.804688,-2.474609,2.880859,0.426514,-0.582520,3.105469,0.839355,1.042969
4,8,1.522461,-2.654297,0.423584,1.816406,2.029297,1.697266,0.608887,-1.333008,-1.022461,...,1.652344,-1.703125,-2.476562,-4.488281,1.402344,0.639160,0.059204,1.250000,2.458984,2.517578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,48438,0.907227,-1.423828,1.901367,2.382812,1.664062,0.900879,-1.924805,-0.843262,-1.282227,...,1.071289,-0.565918,-1.356445,-2.285156,0.418701,1.647461,0.828125,1.503906,2.166016,2.939453
5996,48466,2.138672,-1.450195,0.352051,2.126953,0.429688,-0.171875,-0.984863,-2.195312,-2.027344,...,-0.836914,-1.895508,-2.447266,-2.021484,1.229492,0.562500,-1.253906,2.990234,2.492188,1.022461
5997,48470,1.147461,-2.382812,0.937988,0.431396,1.119141,2.527344,-1.699219,-1.219727,-1.711914,...,0.713379,-0.082031,-1.974609,-0.666992,2.468750,0.973633,1.822266,-0.182739,2.941406,0.713867
5998,48483,0.945801,-1.449219,-0.097046,0.824707,1.160156,1.100586,-0.375732,-2.132812,-0.812012,...,1.913086,0.848145,-1.116211,-1.067383,1.034180,1.029297,-0.780762,0.202637,2.699219,1.937500


In [11]:
sent_vec_df.to_csv('synopsis_vector.csv', index=False)